In [1]:
import pandas as pd
import requests
from sodapy import Socrata
from sqlalchemy import create_engine
from time import time
import argparse
#import psycopg2
from tenacity import retry, wait_exponential, stop_after_attempt
import requests
import pyspark


In [2]:
client = Socrata("data.cityofnewyork.us", "xoIfIdDlHq6gGzxqLqbUeMpsG")

In [3]:
@retry(wait=wait_exponential(multiplier=2, min=2, max=16), stop=stop_after_attempt(5))
def get_data_from_api(client,data_set,limit_rows):
    results = client.get(data_set,limit=limit_rows)
    return results
try:
    #results = client.get("8m42-w767", limit=50)
    results = get_data_from_api(client,"7ym2-wayt",10000)
    print("Connected to API")
    
except requests.exceptions.RequestException as e:
    print(f"Failed to fetch data from API: {e}")

Connected to API


## Pandas

In [4]:
df = pd.DataFrame.from_records(results)

In [5]:
df.head()

,requestid,boro,yr,m,d,hh,mm,vol,segmentid,wktgeom,street,fromst,tost,direction
0,32970,Queens,2021,4,30,2,0,0,149701,POINT (997407.0998491726 208620.92612708386),PULASKI BRIDGE,Newtown Creek Shoreline,Dead end,NB
1,32970,Queens,2021,4,30,2,15,1,149701,POINT (997407.0998491726 208620.92612708386),PULASKI BRIDGE,Newtown Creek Shoreline,Dead end,NB
2,11342,Brooklyn,2012,12,18,8,15,33,20063,POINT (985746.5 167127.4),61 ST,15 AV,16 AV,WB
3,32970,Queens,2021,4,30,2,30,0,149701,POINT (997407.0998491726 208620.92612708386),PULASKI BRIDGE,Newtown Creek Shoreline,Dead end,NB
4,32970,Queens,2021,4,30,2,45,0,149701,POINT (997407.0998491726 208620.92612708386),PULASKI BRIDGE,Newtown Creek Shoreline,Dead end,NB


In [6]:
df.dtypes

requestid    object
boro         object
yr           object
m            object
d            object
hh           object
mm           object
vol          object
segmentid    object
wktgeom      object
street       object
fromst       object
tost         object
direction    object
dtype: object

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col
from pyspark.sql.functions import to_timestamp, to_date
spark = SparkSession.builder.appName("Transformations_NYC_Fire_Incidents").getOrCreate()
print(spark.version)

3.5.0


In [8]:
df = spark.read.json(spark.sparkContext.parallelize([results]))

In [ ]:
df.show()

In [9]:
df.createOrReplaceTempView('Temp_Tbl')

In [22]:
query = spark.sql("SELECT boro,street,sum(vol),yr,m,d,hh,mm FROM Temp_Tbl where yr = 2021 Group By boro,street,yr,m,d,hh,mm order by street,yr,m,d,hh,mm asc;")
query.show()

+------+----------------+--------+----+---+---+---+---+
|  boro|          street|sum(vol)|  yr|  m|  d| hh| mm|
+------+----------------+--------+----+---+---+---+---+
|Queens|JERICHO TURNPIKE|   164.0|2021|  5| 17| 10|  0|
|Queens|JERICHO TURNPIKE|   164.0|2021|  5| 17| 10| 15|
|Queens|JERICHO TURNPIKE|   156.0|2021|  5| 17| 10| 30|
|Queens|JERICHO TURNPIKE|   179.0|2021|  5| 17| 10| 45|
|Queens|JERICHO TURNPIKE|   180.0|2021|  5| 17| 11|  0|
|Queens|JERICHO TURNPIKE|   167.0|2021|  5| 17| 11| 15|
|Queens|JERICHO TURNPIKE|   166.0|2021|  5| 17| 11| 30|
|Queens|JERICHO TURNPIKE|   189.0|2021|  5| 17| 11| 45|
|Queens|JERICHO TURNPIKE|   181.0|2021|  5| 17| 12|  0|
|Queens|JERICHO TURNPIKE|   203.0|2021|  5| 17| 12| 15|
|Queens|JERICHO TURNPIKE|   197.0|2021|  5| 17| 12| 30|
|Queens|JERICHO TURNPIKE|   181.0|2021|  5| 17| 12| 45|
|Queens|JERICHO TURNPIKE|   207.0|2021|  5| 17| 13|  0|
|Queens|JERICHO TURNPIKE|   204.0|2021|  5| 17| 13| 15|
|Queens|JERICHO TURNPIKE|   202.0|2021|  5| 17| 

In [23]:
query = spark.sql("SELECT yr,m,count(m) FROM Temp_Tbl where yr = 2021 Group By yr,m order by yr,m asc;")
query.show()

+----+---+--------+
|  yr|  m|count(m)|
+----+---+--------+
|2021| 10|      46|
|2021|  3|     192|
|2021|  4|     320|
|2021|  5|     390|
+----+---+--------+



In [29]:
@retry(wait=wait_exponential(multiplier=2, min=2, max=16), stop=stop_after_attempt(5))
def get_data_from_api(client,data_set,limit_rows):
    address_results = client.get(data_set,limit=limit_rows)
    return address_results
try:
    #results = client.get("8m42-w767", limit=50)
    address_results = get_data_from_api(client,"uf93-f8nk",10000)
    print("Connected to API")
    
except requests.exceptions.RequestException as e:
    print(f"Failed to fetch data from API: {e}")

Connected to API


In [30]:
address_results

[{'the_geom': {'type': 'Point',
   'coordinates': [-73.86341883800988, 40.68162294566524]},
  'address_id': '207992',
  'bin': '4186824',
  'h_no': '95-32',
  'hyphen_typ': 'Q',
  'side_of_st': '2',
  'borocode': '4',
  'zipcode': '11416',
  'created': '2008-09-23T00:00:00.000',
  'st_name': '75',
  'physicalid': '31963',
  'post_type': 'ST',
  'full_stree': '75 ST',
  ':@computed_region_f5dn_yrer': '46',
  ':@computed_region_yeji_bk3q': '3',
  ':@computed_region_sbqj_enih': '60',
  ':@computed_region_92fq_4b7q': '41'},
 {'the_geom': {'type': 'Point',
   'coordinates': [-73.86603108832932, 40.74156812143741]},
  'address_id': '52411',
  'bin': '4046774',
  'h_no': '50-14',
  'hyphen_typ': 'Q',
  'side_of_st': '2',
  'borocode': '4',
  'zipcode': '11368',
  'created': '2008-09-23T00:00:00.000',
  'st_name': '97',
  'physicalid': '84320',
  'post_type': 'ST',
  'full_stree': '97 ST',
  ':@computed_region_f5dn_yrer': '66',
  ':@computed_region_yeji_bk3q': '3',
  ':@computed_region_sbqj_en

In [32]:
address_df = spark.read.json(spark.sparkContext.parallelize([address_results]))

In [33]:
address_df.createOrReplaceTempView('Temp_Tbl')

In [37]:
query = spark.sql("SELECT borocode,bin from Temp_Tbl where bin = 149701 limit 5")
query.show()

+--------+---+
|borocode|bin|
+--------+---+
+--------+---+



In [34]:
address_df.dtypes

[(':@computed_region_92fq_4b7q', 'string'),
 (':@computed_region_f5dn_yrer', 'string'),
 (':@computed_region_sbqj_enih', 'string'),
 (':@computed_region_yeji_bk3q', 'string'),
 ('address_id', 'string'),
 ('bin', 'string'),
 ('borocode', 'string'),
 ('created', 'string'),
 ('full_stree', 'string'),
 ('h_no', 'string'),
 ('hn_rng', 'string'),
 ('hn_rng_suf', 'string'),
 ('hno_suffix', 'string'),
 ('hyphen_typ', 'string'),
 ('modified', 'string'),
 ('physicalid', 'string'),
 ('post_direc', 'string'),
 ('post_modif', 'string'),
 ('post_type', 'string'),
 ('pre_direct', 'string'),
 ('pre_modifi', 'string'),
 ('pre_type', 'string'),
 ('side_of_st', 'string'),
 ('special_co', 'string'),
 ('st_name', 'string'),
 ('the_geom', 'struct<coordinates:array<double>,type:string>'),
 ('zipcode', 'string')]